# Heston-Bates 1996

## Vanilla Option Pricing 

In [31]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

In [32]:
def heston_bates_charfunc(u, params):
    S0    = params["S0"]
    K     = params["K"]
    r     = params["r"]
    q     = params["q"]
    λ     = params["lambda"]
    μJ    = params["muJ"]
    δJ    = params["deltaJ"]
    v0    = params["v0"]
    kappa = params["kappa"]
    theta = params["theta"]
    sigma = params["sigma"]
    rho   = params["rho"]
    T     = params["T"]

    iu = 1j * u

    # —— Heston core (unchanged) ——
    alpha = -0.5 * (u**2 + iu)
    beta  = kappa - rho * sigma * iu
    gamma = 0.5 * sigma**2
    d     = np.sqrt(beta**2 - 4.0 * alpha * gamma)
    g     = (beta - d) / (beta + d)
    exp_dT = np.exp(-d * T)
    C = ((kappa*theta/gamma)*((beta-d)*T - 2*np.log((1 - g*exp_dT)/(1 - g))))
    D = ((beta-d)/gamma) * ((1-exp_dT)/(1-g*exp_dT))


    # —— Adjust drift for jumps ——
    r_tilde = r - q - λ*(np.exp(μJ + 0.5*δJ**2) - 1)
    drift  = r_tilde * iu * T

    # —— Bates jump‐part ——
    jump_cf   = np.exp(iu*μJ - 0.5*(u**2)*δJ**2)
    jump_term = λ * T * (jump_cf - 1)

    C = drift + C + jump_term

    return np.exp(C + D * v0 + iu * np.log(S0 * np.exp(-q * T)))

def heston_price_call_fft_bates(params, N=10000, U_max=1000):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]

    lnK = np.log(K)
    phi_u     = heston_bates_charfunc(u, params)
    phi_u_im1 = heston_bates_charfunc(u - 1j, params)
    phi_im1   = heston_bates_charfunc(-1j, params)

    integrand_P1 = np.real(np.exp(-1j * u * lnK) * phi_u_im1 / (1j * u * phi_im1))
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1)
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put  = K * np.exp(-r * T) * (1 - P2) - S0 * np.exp(-q * T) * (1 - P1)

    return call, put


In [33]:
params = {
    "S0": 100,
    "K" : 135,
    "r":  0.02,
    "q": 0.01,
    "v0": 0.08,
    "lambda": 0.30,
    "muJ": 0.02,
    "deltaJ": 0.2,
    "kappa": 1.12,
    "theta": 0.05,
    "sigma": 0.50,
    "rho": -0.25,
    "T": 1.2
}

call,put = heston_price_call_fft_bates(params, N=10000, U_max=1000)
print(f"Call Price: {call}")
print(f"Put Price: {put}")

Call Price: 5.928196087654005
Put Price: 38.91959561877874


## Market Data

In [34]:
import pandas as pd

excel_file = "market_data.xlsx"

wb = pd.read_excel(excel_file,sheet_name='NDX-2014' ,header=None,)


header_row_idx = 6
start_col = 2
end_col = 13               
n_maturities = 14

raw_header = wb.iloc[header_row_idx, start_col:end_col].tolist()
raw_data   = wb.iloc[
    header_row_idx+1 : header_row_idx+1+n_maturities,
    start_col : end_col
]


df = pd.DataFrame(
    raw_data.values[:, 1:],          
    index=raw_data.values[:, 0],    
    columns=raw_header[1:],         
)


df.columns = [f"{x:.1%}" for x in df.columns.astype(float)]


print(df.head())

S0 = wb.iat[3, 4]

r = wb.iat[1, 10]

q = wb.iat[2, 10]

       80.0%     90.0%     95.0%      97.5%   100.0%     102.5%    105.0%  \
T   3036.592  3416.166  3605.953  3700.8465  3795.74  3890.6335  3985.527   
1M     25.29     19.15     15.35       13.7    12.25      11.04     10.71   
2M     24.28     18.46     15.81      14.58    13.43      12.39     11.67   
3M     23.03     18.08     15.96      14.96     14.0       13.1     12.38   
6M     21.93     18.24     16.61      15.85    15.13      14.47     13.86   

      110.0%    120.0%  
T   4175.314  4554.888  
1M     11.46     13.56  
2M     11.52     12.55  
3M     11.83     12.06  
6M     12.87      12.4  


## Calibration

In [35]:
# --- 1) Black–Scholes call price (inchangé) ---

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

# --- 2) Extraction des données du DataFrame ------------------------------------------------

# On suppose que votre df ressemble à ceci (cf. message précédent) :
#   - index : maturities (str) avec une ligne 'T' qui contient les strikes
#   - colonnes : niveaux de strike en pourcentage ('80%', '90%', …, '120%')
#   - df.loc['T', k] = strike absolu ; df.loc[maturity, k] = vol implicite



# Conversion des labels de maturités en années
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(label)

# Construction d’une liste de points marché (T, K, prix_call)
market_data = []
strikes_abs = df.loc['T']  # Série indexée par '80%',…,'120%'
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K      = strikes_abs[pct]
        vol_imp= df.loc[mat, pct]/100.0  # en pourcentage
        price  = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data, 
                       dtype=[('T', float), ('K', float), ('price', float)])

# --- 3) Dépaquetage des paramètres pour Heston–Bates ---

def pack_params_bates(x):
    """Dépaquète x en dict de params pour Heston–Bates."""
    return {
        "S0": S0,
        "r":     r,
        "q":     q,
        "v0":    x[0],
        "kappa": x[1],
        "theta": x[2],
        "sigma": x[3],
        "rho":   x[4],
        "lambda":x[5],
        "muJ":   x[6],
        "deltaJ":x[7],
    }

# --- 4) Fonction objectif (résidus modèle – marché) ---

def residuals_bates(x):
    params_base = pack_params_bates(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = heston_price_call_fft_bates(params,
                                                   N=1000,
                                                   U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)

# --- 5) Calibration par moindres carrés ---

# 5.1) Initial guess pour [v0, κ, θ, σ, ρ, λ, μJ, δJ]
x0 = np.array([
    0.04,    # v0
    1.0,     # kappa
    0.04,    # theta
    0.5,     # sigma
    -0.5,    # rho
    0.1,     # lambda (intensité de saut)
    -0.02,   # muJ   (moyenne du saut)
    0.1,     # deltaJ (écart‑type du saut)
])

# 5.2) Bornes : λ, δJ ≥ 0 ; ρ∈(−1,1) ; μJ non borné ou modérément borné
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999, 0.0,   -1.0,   1e-4]
ub = [2.0,   10.0,   2.0,   5.0,   0.999,  5.0,    1.0,    5.0]

opt = least_squares(
    residuals_bates,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=200
)

# 5.3) Affichage des résultats
(v0_fit, kappa_fit, theta_fit, sigma_fit,
 rho_fit, lambda_fit, muJ_fit, deltaJ_fit) = opt.x

print("Paramètres calibrés Heston–Bates :")
print(f" v0     = {v0_fit:.4f}")
print(f" kappa  = {kappa_fit:.4f}")
print(f" theta  = {theta_fit:.4f}")
print(f" sigma  = {sigma_fit:.4f}")
print(f" rho    = {rho_fit:.4f}")
print(f" lambda = {lambda_fit:.4f}")
print(f" muJ    = {muJ_fit:.4f}")
print(f" deltaJ = {deltaJ_fit:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.7530e+05                                    7.25e+06    
       1              2         8.9304e+04      6.86e+05       5.13e-01       1.03e+06    
       2              3         1.9433e+04      6.99e+04       3.22e-01       1.60e+05    
       3              4         8.4104e+03      1.10e+04       1.21e-01       6.65e+04    
       4              5         6.7995e+03      1.61e+03       2.48e-01       1.41e+05    
       5              6         6.0129e+03      7.87e+02       1.28e-01       1.45e+05    
       6              7         4.6471e+03      1.37e+03       1.02e-01       1.32e+05    
       7             10         4.2568e+03      3.90e+02       7.07e-02       3.40e+05    
       8             11         3.8850e+03      3.72e+02       1.28e-01       2.33e+06    
       9             13         3.3028e+03      5.82e+02       4.12e-02       3.05e+04    

In [36]:
# Calibration test 
# Test de la calibration
# On compare les prix Heston et Black-Scholes
print("Prix Black-Sholes, M=1Y, S=100%:", bs_call_price(S0,S0,r,q,0.1639,1.0))
params = {
    "S0": S0,
    "K" : S0,
    "r":  r,
    "q": q,
    "T": 1.0,
    "v0" : v0_fit,
    "kappa" :kappa_fit,
    "theta" : theta_fit,
    "sigma"  : sigma_fit,
    "rho"    : rho_fit,
    "lambda" : lambda_fit,
    "muJ" : muJ_fit,
    "deltaJ": deltaJ_fit,
}

print("Prix Heston, M=1Y, S=100%:", heston_price_call_fft_bates(params, N=1000, U_max=2000)[0])

Prix Black-Sholes, M=1Y, S=100%: 242.4783779478173
Prix Heston, M=1Y, S=100%: 241.86001307786046
